In [1]:
import json
from web3 import Web3
from pprint import pprint

# 1. Setting up

For local blockchain:

In [2]:
# Addresses and private keys
addr1 = "0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266"
key1 = "0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80"

addr2 = "0x70997970C51812dc3A010C7d01b50e0d17dc79C8"
key2 = "0x59c6995e998f97a5a0044966f0945389dc9e86dae88c7a8412f4603b6b78690d"

# Connect to the blockchain
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
DM_address = "0xe7f1725E7734CE288F8367e1Bb143E90bb3F0512"
PLT_adress = "0x5FbDB2315678afecb367f032d93F642f64180aa3"

For Sepolia blockchain:

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
owner_addr = '0x0E8505C4751F05F19076ed4DF9187971976410e3'
addr1 = config['addresses']['addr1']
key1 = config['private_keys']['key1']

w3 = Web3(Web3.HTTPProvider("https://sepolia.infura.io/v3/ca745db3f9cf4dcdbc16110c9c83c55c"))

DM_address = "0x3d9777110C32508a68f152A70F1d900ac3642648"
PLT_adress = "0x8ff9B0F3632447e7eFfbEBD4C571CE939d3aCe4C"

In [6]:
# Extract the ABI and bytecode
with open("./artifacts/contracts/ProLongToken.sol/ProLongToken.json") as PLTjson_:
    PLTjson = json.load(PLTjson_)
with open("./artifacts/contracts/DataMarket.sol/DataMarket.json") as DMjson_:
    DMjson = json.load(DMjson_)

PLT_abi = PLTjson["abi"]
PLT_bytecode = PLTjson["bytecode"]

DM_abi = DMjson["abi"]
DM_bytecode = DMjson["bytecode"]

PLToken = w3.eth.contract(address=PLT_adress, abi=PLT_abi)
DataMarket = w3.eth.contract(address=DM_address, abi=DM_abi)
chain_id = w3.eth.chain_id

# 2. Functions which could be used if needed

view functions are called like this:

`contract.functions.functionName(functionArgs).call()`

In [7]:
print(f"Initial balance of addr_1: {PLToken.functions.balanceOf(addr1).call()}")
# print(f"Initial balance of addr_2: {PLToken.functions.balanceOf(addr2).call()}")

Initial balance of addr_1: 1000000000000000000000000


In [ ]:
PLToken.all_functions()

[<Function allowance(address,address)>,
 <Function approve(address,uint256)>,
 <Function balanceOf(address)>,
 <Function decimals()>,
 <Function name()>,
 <Function symbol()>,
 <Function totalSupply()>,
 <Function transfer(address,uint256)>,
 <Function transferFrom(address,address,uint256)>]

In [ ]:
DataMarket.all_functions()

[<Function buyFile(bytes32)>,
 <Function buyerFiles(address,uint256)>,
 <Function deleteFile(bytes32)>,
 <Function fileIdToHash(uint256)>,
 <Function files(bytes32)>,
 <Function getBuyerFiles(address)>,
 <Function getFileInfo(bytes32)>,
 <Function getSellerFiles(address)>,
 <Function isBuyer(bytes32,address)>,
 <Function sellerFiles(address,uint256)>,
 <Function token()>,
 <Function uploadFile(uint256,bytes32)>,
 <Function uploadedFiles(uint256)>]

# 3. Implementations for main transaction functions

In [ ]:
def build_and_send_transaction(contract, function_name:str, addr_from:str, private_key:str, *args, **kwargs):
    """template for all transactions (functions that modify blockchain state). This includes transfer, upload, buy functions

    Args:
        contract (web3._utils.datatypes.Contract): PLToken or DataMarket
        function_name (str): function name from blockchain
        addr_from (str): address of the sender
        private_key (str): private key of the sender. It is used to sign the transaction
    """
    # Get the latest transaction nonce
    nonce = w3.eth.get_transaction_count(addr_from)

    # Dynamically access the function based on the provided function name
    function_to_call = getattr(contract.functions, function_name)

    # Build the transaction object
    transaction = function_to_call(*args).build_transaction(
        {
            "chainId": chain_id,
            "gasPrice": w3.eth.gas_price,
            "from": addr_from,
            "nonce": nonce,
            **kwargs,
        }
    )

    # Sign the transaction
    signed_txn = w3.eth.account.sign_transaction(transaction, private_key=private_key)
    # print("Deploying Contract!")

    # Send the transaction
    tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)

    # Wait for the transaction to be mined and get the transaction receipt
    # print("Waiting for transaction to finish...")
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

    # print(f"Done! Transaction executed successfully with hash: {tx_hash}")

In [ ]:
def transfer(addr_from: str, addr_to: str, private_key: str, Ntokens: int):
    """transfer PLTokens from one address to another. It's needed after deploying contract to share tokens from one account to another

    Args:
        addr_from (str): sender address
        addr_to (str): reciever address
        private_key (str): sender private key
        Ntokens (int): number of tokens to send
    """
    print("Initializing transfer...")
    build_and_send_transaction(
        PLToken, "transfer", addr_from, private_key, addr_to, Ntokens
    )
    print("Done")
    print("#"*20)

def upload(addr: str, private_key: str, cost: int, hash_: str) -> None:
    """uploads file info on blockchain

    Args:
        addr (str): seller's address
        private_key (str): seller's private key
        cost (int): file's cost
        hash_ (str): file's hash. It is used as a unique identifier. Example: "0xa589559b0443b454b7f0dd10565a985bec8a9122d242877bcc42dad71bf8588c"
    """
    print("Initializing upload...")
    build_and_send_transaction(DataMarket, "uploadFile", addr, private_key, cost, hash_)
    print("Done")
    print("#"*20)


def getFileInfo(hash: str) -> set:
    """returns a file info

    Args:
        hash (str): file's hash. It is used as a unique identifier. Example: "0xa589559b0443b454b7f0dd10565a985bec8a9122d242877bcc42dad71bf8588c"

    Returns:
        set: fileInfo
    """
    print("Getting file info...")
    info = DataMarket.functions.getFileInfo(hash).call()
    fileInfo = {
        "address": info[0],
        "cost": info[1],
        "fileId": info[2],
        "isAvailable": info[3],
    }
    print("#"*20)
    return fileInfo


def buy(hash_:str, addr_from: str, private_key:str)->str:
    """function for buying a file. It returns buyer's address if the transaction was successful

    Args:
        hash_ (str): file's hash. It is used as a unique identifier. Example: "0xa589559b0443b454b7f0dd10565a985bec8a9122d242877bcc42dad71bf8588c"
        addr_from (str): buyer's address
        private_key (str): buyer's private key

    Returns:
        str: buyer's address
    """

    # TODO check if file is available
    # TODO check if buyer has enough PLTokens
    # TODO check if buyer already has this file
    print("Buying file...")
    # get file cost and seller ( it should get it from database I suppose )
    seller = DataMarket.functions.getFileInfo(hash_).call()[0]
    cost = DataMarket.functions.getFileInfo(hash_).call()[1]
    # approve the transfer of money from buyers account
    print("Approving the transaction...")
    build_and_send_transaction(PLToken, "approve", addr_from, private_key, DM_address, cost)
    # initiate transfer of tokens
    print("Initiating the transaction...")
    build_and_send_transaction(DataMarket, "buyFile", addr_from, private_key, hash_)
    # listen to the event
    print("Checking the transaction...")
    log = DataMarket.events.FileSold.get_logs(fromBlock=w3.eth.block_number)[0]
    print("Log of FileSold event: \n", log)
    assert log.args.seller == seller, "Seller address does not match"
    assert log.args.buyer == addr2, "Buyer address does not match"
    # hash of the file in system transforms to bytes interpetation,
    # that's why we need to convert it to hex
    assert "0x"+log.args.hash.hex() == hash_, "File's hash does not match"
    print("Done")
    print("#"*20)
    return addr2


def deleteFile(hash_:str, addr_from: str, private_key:str):
    """It can be used for deleting a file after uploading it

    Args:
        hash_ (str): files hash
        addr_from (str): seller address
        private_key (str): seller private key
    """
    # TODO it's not implemented completely inside contract.
    print("Deleting file...")
    build_and_send_transaction(DataMarket, "deleteFile", addr_from, private_key, hash_)
    print("Done")
    print("#"*20)

# 4. Actually using functions ( example )

In [6]:
# random hash for an example

# import codecs
# from randomHash import getRandomHash
# hash_ = getRandomHash()
# print(hash_)

hash_ = "0xa589559b0443b454b7f0dd10565a985bec8a9122d242877bcc42dad71bf8588c"

In [10]:
#1. Transfer some tokens to second address
transfer(addr1, addr2, key1, 1000)
#2. Upload file from first address
upload(addr1, key1, 100, hash_)
#3. Get file info
pprint(getFileInfo(hash_))
#4. Buy file from second address
buyers_address = buy(hash_, addr2, key2)
print(f"Buyer address: {buyers_address}")

Initializing transfer...
Done
####################
Initializing upload...
Done
####################
Getting file info...
####################
{'address': '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
 'cost': 100,
 'fileId': 0,
 'isAvailable': True}
Buying file...
Approving the transaction...
Initiating the transaction...
Checking the transaction...
Log of FileSold event: 
 AttributeDict({'args': AttributeDict({'seller': '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266', 'buyer': '0x70997970C51812dc3A010C7d01b50e0d17dc79C8', 'hash': b'\xa5\x89U\x9b\x04C\xb4T\xb7\xf0\xdd\x10VZ\x98[\xec\x8a\x91"\xd2B\x87{\xccB\xda\xd7\x1b\xf8X\x8c'}), 'event': 'FileSold', 'logIndex': 1, 'transactionIndex': 0, 'transactionHash': HexBytes('0x9db83e82bee98018e2949aad2176d2a65b0f226136c967115e4ff21bc55d73da'), 'address': '0xe7f1725E7734CE288F8367e1Bb143E90bb3F0512', 'blockHash': HexBytes('0xcfce5f2493c4f717b0398f7de4107260fbe6befaf051867688013751c44b8726'), 'blockNumber': 6})
Done
####################
Buyer addre

In [11]:
print(addr2 == buyers_address)

True


In [12]:
print("Sellers files:", DataMarket.functions.getSellerFiles(addr1).call())
deleteFile(hash_, addr1, key1)
print("Sellers files after deleting:", DataMarket.functions.getSellerFiles(addr1).call())

Sellers files: [b'\xa5\x89U\x9b\x04C\xb4T\xb7\xf0\xdd\x10VZ\x98[\xec\x8a\x91"\xd2B\x87{\xccB\xda\xd7\x1b\xf8X\x8c']
Deleting file...
Done
####################
Sellers files after deleting: []
